# Intelligent Agents #

This notebook uses implementations from [agents.py](https://github.com/aimacode/aima-python/blob/master/agents.py) module. Let's start by importing everything from agents module.

![Robot Wave](https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExNHJ1eHQ1aWxqM2xyZzA0a3Z6bDJyMmFweGsxeHBhM3Z5NHUwenZraSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/5k5vZwRFZR5aZeniqb/giphy.gif)

In [1]:
from agents import *
from notebook import psource

## CONTENTS
    
* Overview
* Agent
* Environment
* 1: Vaccum cleaner
* Blind Dog
* Energetic Blind Dog


## OVERVIEW

An agent,is anything that can perceive its <b>environment</b> through sensors, and act upon that environment through actuators based on its <b>agent program</b>. This can be a dog, a robot, or even you. As long as you can perceive the environment and act on it, you are an agent. This notebook will explain how to implement a simple agent, create an environment, and implement a program that helps the agent act on the environment based on its percepts.

## AGENT

Let us now see how we define an agent. Run the next cell to see how `Agent` is defined in agents module.

In [2]:
psource(Agent)

The `Agent` has two methods.
* `__init__(self, program=None)`: The constructor defines various attributes of the Agent. These include

    * `alive`: which keeps track of whether the agent is alive or not 
    
    * `bump`: which tracks if the agent collides with an edge of the environment (for eg, a wall in a park)
    
    * `holding`: which is a list containing the `Things` an agent is holding, 
    
    * `performance`: which evaluates the performance metrics of the agent 
    
    * `program`: which is the agent program and maps an agent's percepts to actions in the environment. If no implementation is provided, it defaults to asking the user to provide actions for each percept.
    
* `can_grab(self, thing)`: Is used when an environment contains things that an agent can grab and carry. By default, an agent can carry nothing.

## ENVIRONMENT
Now, let us see how environments are defined. Running the next cell will display an implementation of the abstract `Environment` class.

In [3]:
psource(Environment)

`Environment` class has lot of methods! But most of them are incredibly simple, so let's see the ones we'll be using in this notebook.

* `thing_classes(self)`: Returns a static array of `Thing` sub-classes that determine what things are allowed in the environment and what aren't

* `add_thing(self, thing, location=None)`: Adds a thing to the environment at location

* `run(self, steps)`: Runs an environment with the agent in it for a given number of steps.

* `is_done(self)`: Returns true if the objective of the agent and the environment has been completed

The next two functions must be implemented by each subclasses of `Environment` for the agent to recieve percepts and execute actions 

* `percept(self, agent)`: Given an agent, this method returns a list of percepts that the agent sees at the current time

* `execute_action(self, agent, action)`: The environment reacts to an action performed by a given agent. The changes may result in agent experiencing new percepts or other elements reacting to agent input.

## 1: Vacuum-World Agent (Table-Driven Agent)

Run and understand the following implementation of a table-driven vacuum-world agent


In [4]:
from agents import Agent,Thing
import random
import numbers

In [ ]:
class Environment:

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        raise NotImplementedError

    def execute_action(self, agent, action):
        raise NotImplementedError

    def default_location(self, thing):
        return None

    def exogenous_change(self):
        pass

    def is_done(self):
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        print("Status of enviorment is: ",self.status)
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
               
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            if self.is_done():
                return
            print()
            self.step()


    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)
        print("location of agent is: ",thing.location)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

In [6]:
class TrivialVacuumEnvironment(Environment):
    """This environment has two locations, A and B. Each can be Dirty or Clean."""

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty'])}
        self.dirty_timer = {loc_A: 0, loc_B: 0}

    def thing_classes(self):
        return [TableDrivenVacuumAgent]

    def percept(self, agent):
        print("Agent percepts: ", agent.location, self.status[agent.location])
        return agent.location, self.status[agent.location]

    def execute_action(self, agent, action):
        print(f"Agent is executing {action} at {agent.location}")
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
                self.status[agent.location] = 'Clean'
                self.dirty_timer[agent.location] = 0
        print(f"Agent's performance is {agent.performance}")

    def exogenous_change(self):
        """Make rooms dirty again after 1 or 2 time steps."""
        for loc in [loc_A, loc_B]:
            if self.status[loc] == 'Clean':
                self.dirty_timer[loc] += 1
                if self.dirty_timer[loc] > random.choice([1, 2]):
                    self.status[loc] = 'Dirty'
                    self.dirty_timer[loc] = 0

    def default_location(self, thing):
        location = random.choice([loc_A, loc_B])
        return location


In [7]:
def TableDrivenAgentProgram(table):
    """
    [Figure 2.7]
    This agent selects an action based on the percept sequence.
    It is practical only for tiny domains.
    To customize it, provide as table a dictionary of all
    {percept_sequence:action} pairs.
    """
    percepts = []

    def program(percept):
        percepts.append(percept)
        action = table.get(tuple(percepts))
        if action is None:
            print("****Error: No action given in table for this percept****")
        return action

    return program

In [8]:

def TableDrivenVacuumAgent():
    """Tabular approach towards vacuum world as mentioned in [Figure 2.3]
    >>> agent = TableDrivenVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """

    loc_A = (0,0)
    loc_B = (1,0)

    #Edit this table 
    table = {
            #All one percept possibilities
            ((loc_A, 'Clean'),): 'Right',
             ((loc_A, 'Dirty'),): 'Suck',
             ((loc_B, 'Clean'),): 'Left',
             ((loc_B, 'Dirty'),): 'Suck',

            #All two percept possibilities
             ((loc_A, 'Dirty'), (loc_A, 'Clean')): 'Right',
             ((loc_A, 'Clean'), (loc_B, 'Dirty')): 'Suck',
             ((loc_B, 'Clean'), (loc_A, 'Dirty')): 'Suck',
             ((loc_B, 'Dirty'), (loc_B, 'Clean')): 'Left',

             #Complete the list of three percept possibilities
            ((loc_A, 'Dirty'), (loc_A, 'Clean'), (loc_B, 'Dirty')): 'Suck',
            ((loc_A, 'Dirty'), (loc_A, 'Clean'), (loc_B, 'Clean')): 'Left',
            

            #add possibilites for 4 percepts here
            ((loc_A, 'Clean'), (loc_B, 'Clean'), (loc_A, 'Dirty'), (loc_A, 'Clean')): 'Right',
            ((loc_A, 'Dirty'), (loc_A, 'Clean'), (loc_B, 'Dirty'), (loc_B, 'Clean')): 'Left',
            ((loc_B, 'Dirty'), (loc_B, 'Clean'), (loc_A, 'Dirty'), (loc_A, 'Clean')): 'Right',
            ((loc_B, 'Clean'), (loc_A, 'Dirty'), (loc_A, 'Clean'), (loc_B, 'Dirty')): 'Suck',
        
        }
    
    return Agent(TableDrivenAgentProgram(table))

In [9]:
agent = TableDrivenVacuumAgent() #initializes Table driven vaccuum agent
env = TrivialVacuumEnvironment() #Builds the vacuum environment

env.add_thing(agent)

env.run(6) #Modify the number inside run

location of agent is:  (0, 0)

Status of enviorment is:  {(0, 0): 'Clean', (1, 0): 'Clean'}
Agent percepts:  (0, 0) Clean
Agent is executing Right at (0, 0)
Agent's performance is -1

Status of enviorment is:  {(0, 0): 'Clean', (1, 0): 'Clean'}
Agent percepts:  (1, 0) Clean
****Error: No action given in table for this percept****
Agent is executing None at (1, 0)
Agent's performance is -1

Status of enviorment is:  {(0, 0): 'Dirty', (1, 0): 'Clean'}
Agent percepts:  (1, 0) Clean
****Error: No action given in table for this percept****
Agent is executing None at (1, 0)
Agent's performance is -1

Status of enviorment is:  {(0, 0): 'Dirty', (1, 0): 'Dirty'}
Agent percepts:  (1, 0) Dirty
****Error: No action given in table for this percept****
Agent is executing None at (1, 0)
Agent's performance is -1

Status of enviorment is:  {(0, 0): 'Dirty', (1, 0): 'Dirty'}
Agent percepts:  (1, 0) Dirty
****Error: No action given in table for this percept****
Agent is executing None at (1, 0)
Agent's

## SIMPLE AGENT AND ENVIRONMENT

Let's begin by using the `Agent` class to creating our first agent - a blind dog.

Run and understand the following code.

In [10]:
class BlindDog(Agent):
    def eat(self, thing):
        print("Dog: Ate food at {}.".format(self.location))
            
    def drink(self, thing):
        print("Dog: Drank water at {}.".format( self.location))

dog = BlindDog()

Can't find a valid program for BlindDog, falling back to default.


What we have just done is create a dog who can only feel what's in his location (since he's blind), and can eat or drink. Let's see if he's alive...

In [11]:
print(dog.alive)

True


![Cool dog](https://gifgun.files.wordpress.com/2015/07/wpid-wp-1435860392895.gif)
This is our dog. How cool is he? Well, he's hungry and needs to go search for food. For him to do this, we need to give him a program. But before that, let's create a park for our dog to play in.

### ENVIRONMENT - Park

A park is an example of an environment because our dog can perceive and act upon it. The <b>Environment</b> class is an abstract class, so we will have to create our own subclass from it before we can use it.

In [12]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles
    def status(self):
        """Custom status for debugging"""
        return {
            "agents": [(a.__class__.__name__, a.location) for a in self.agents],
            "things": [(t.__class__.__name__, getattr(t, "location", None)) for t in self.things if not isinstance(t, Agent)]
        }


### PROGRAM - BlindDog
Now that we have a <b>Park</b> Class, we re-implement our <b>BlindDog</b> to be able to move down and eat food or drink water only if it is present.


In [13]:
class BlindDog(Agent):
    location = 1
    
    def movedown(self):
        self.location += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Now its time to implement a <b>program</b> module for our dog. A program controls how the dog acts upon its environment. Our program will be very simple, and is shown in the table below.
<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>move down</td>
   </tr>
        
</table>

In [14]:
def program(percepts):
    '''Returns an action based on the dog's percepts'''
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    return 'move down'

Let's now run our simulation by creating a park with some food, water, and our dog.

In [15]:
park = Park()
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, 1)
park.add_thing(dogfood, 5)
park.add_thing(water, 7)

park.run(5)

location of agent is:  1
location of agent is:  5
location of agent is:  7

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 1

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 2

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 3

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 4

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog ate Food at location: 5


Notice that the dog moved from location 1 to 4, over 4 steps, and ate food at location 5 in the 5th step.

Let's continue this simulation for 5 more steps.

In [16]:
park.run(6)


Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 5

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 6

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog drank Water at location: 7


Perfect! Note how the simulation stopped after the dog drank the water - exhausting all the food and water ends our simulation, as we had defined before. Let's add some more water and see if our dog can reach it.

In [17]:
park.add_thing(water, 15)
park.run(10)

location of agent is:  15

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 7

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 8

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 9

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 10

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 11

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x00000215C40C6A50>>
BlindDog decided to move down at location: 12

Status of enviorment is:  <bound method Park.status of <__main__.Park object at 0x000002

Above, we learnt to implement an agent, its program, and an environment on which it acts. However, this was a very simple case. Let's try to add complexity to it by creating a 2-Dimensional environment!


## AGENTS IN A 2D ENVIRONMENT

For us to not read so many logs of what our dog did, we add a bit of graphics while making our Park 2D. To do so, we will need to make it a subclass of <b>GraphicEnvironment</b> instead of Environment. Parks implemented by subclassing <b>GraphicEnvironment</b> class adds these extra properties to it:

 - Our park is indexed in the 4th quadrant of the X-Y plane.
 - Every time we create a park subclassing <b>GraphicEnvironment</b>, we need to define the colors of all the things we plan to put into the park. The colors are defined in typical [<b>RGB digital 8-bit format</b>](https://en.wikipedia.org/wiki/RGB_color_model#Numeric_representations), common across the web.
 - Fences are added automatically to all parks so that our dog does not go outside the park's boundary - it just isn't safe for blind dogs to be outside the park by themselves! <b>GraphicEnvironment</b> provides `is_inbounds` function to check if our dog tries to leave the park.
 
First let us try to upgrade our 1-dimensional `Park` environment by just replacing its superclass by `GraphicEnvironment`. 

In [18]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Bone(Thing):
    pass

class Park2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "play":
            items = self.list_things_at(agent.location, tclass=Bone)
            if items:
                item = items[0]
                if agent.play_with_bone(item):
                    print(f'{str(agent)[1:-1]} is playing with {item} at location: {agent.location}')
                    # Note: bone is not deleted – dog can play forever
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

class BlindDog(Agent):
    location = [0,1] # change location to a 2d value
    direction = Direction("down") # variable to store the direction our dog is facing
    
    def movedown(self):
        self.location[1] += 1
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False

Now let's test this new park with our same dog, food and water. We color our dog with a nice red and mark food and water with orange and blue respectively.

In [19]:
park = Park2D(5,20, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)} ) # park width is set to 5, and height to 20
dog = BlindDog(program)
dogfood = Food()
water = Water()

park.add_thing(dog, [0,1])
park.add_thing(dogfood, [0,5])
park.add_thing(water, [0,7])
morewater = Water()
park.add_thing(morewater, [0,15])
print("BlindDog starts at (1,1) facing downwards, lets see if he can find any food!")
park.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


Adding some graphics was a good idea! We immediately see that the code works, but our blind dog doesn't make any use of the 2 dimensional space available to him. Let's make our dog more energetic so that he turns and moves forward, instead of always moving down. In doing so, we'll also need to make some changes to our environment to be able to handle this extra motion.

### PROGRAM - EnergeticBlindDog

Let's make our dog turn or move forwards at random - except when he's at the edge of our park - in which case we make him change his direction explicitly by turning to avoid trying to leave the park. However, our dog is blind so he wouldn't know which way to turn - he'd just have to try arbitrarily.

<table>
    <tr>
        <td><b>Percept:</b> </td>
        <td>Feel Food </td>
        <td>Feel Water</td>
        <td>Feel Nothing</td>
   </tr>
   <tr>
       <td><b>Action:</b> </td>
       <td>eat</td>
       <td>drink</td>
       <td>
       <table>
           <tr>
               <td><b>Remember being at Edge : </b></td>
               <td>At Edge</td>
               <td>Not at Edge</td>
           </tr>
           <tr>
               <td><b>Action : </b></td>
               <td>Turn Left / Turn Right <br> ( 50% - 50% chance )</td>
               <td>Turn Left / Turn Right / Move Forward <br> ( 25% - 25% - 50% chance )</td>
           </tr>
       </table>
       </td>
   </tr>
        
</table>

In [20]:
from random import choice

class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
    def play_with_bone(self, thing):
        if isinstance(thing, Bone):
            self.has_bone = True
            print("Dog is happily playing with the bone!")
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
        
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
        elif isinstance(p, Bone):
            return 'play'
        if isinstance(p,Bump): # then check if you are at an edge and have to turn
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'
    

### ENVIRONMENT - Park2D

We also need to modify our park accordingly, in order to be able to handle all the new actions our dog wishes to execute. Additionally, we'll need to prevent our dog from moving to locations beyond our park boundary - it just isn't safe for blind dogs to be outside the park by themselves.

In [21]:
class Park2D(GraphicEnvironment):
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        loc = agent.location.copy()
        if agent.direction.direction == Direction.R:
            loc[0] += 1  
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1

        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things

    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "play":
            items = self.list_things_at(agent.location, tclass=Bone)
            if items:
                item = items[0]
                if agent.play_with_bone(item):
                    print(f'{str(agent)[1:-1]} is playing with {item} at location: {agent.location}')
                    # Note: bone is not deleted – dog can play forever
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


Now that our park is ready for the 2D motion of our energetic dog, lets test it!

In [22]:
park = Park2D(3, 3, color={
    'EnergeticBlindDog': (200, 0, 0),
    'Water': (0, 200, 200),
    'Food': (230, 115, 40),
    'Bone': (150, 75, 0)
})
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
bone = Bone()
park.add_thing(bone, [2,1]) 

park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
morefood = Food()
park.add_thing(morewater, [1,0])
park.add_thing(morefood, [2,0])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")
park.run(20)

,,
,,
,,


In [23]:
import random
#from agents import Agent, Thing, Direction, Bump, Food, Water, Bone, GraphicEnvironment

class EnergeticBlindDog(Agent):
    def __init__(self, program=None):
        super().__init__(program)
        self.location = [0, 1]
        self.direction = Direction("down")
        self.has_bone = False

    def moveforward(self, success=True):
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1

    def turn(self, d):
        self.direction = self.direction + d

    def eat(self, thing):
        return isinstance(thing, Food)

    def drink(self, thing):
        return isinstance(thing, Water)

    def play_with_bone(self, thing):
        if isinstance(thing, Bone):
            self.has_bone = True
            print("Dog is happily playing with the bone!")
            return True
        return False

def program(percepts):
    bump_detected = False
    for p in percepts:
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
        elif isinstance(p, Bone):
            return 'play'
        elif isinstance(p, Bump):
            bump_detected = True
    if bump_detected:
        return random.choice(['turnright', 'turnleft'])
    else:
        return random.choice(['turnright', 'turnleft', 'moveforward'])

class Park2D(GraphicEnvironment):
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        loc = agent.location.copy()
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1

        if not self.is_inbounds(loc):
            things.append(Bump())
        return things

    def execute_action(self, agent, action):
        if action == 'turnright':
            print(f'{str(agent)[1:-1]} decided to {action} at location: {agent.location}')
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print(f'{str(agent)[1:-1]} decided to {action} at location: {agent.location}')
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print(f'{str(agent)[1:-1]} decided to move {agent.direction.direction}wards at location: {agent.location}')
            agent.moveforward()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if items and agent.eat(items):
                print(f'{str(agent)[1:-1]} ate {str(items)[1:-1]} at location: {agent.location}')
                self.delete_thing(items)
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if items and agent.drink(items):
                print(f'{str(agent)[1:-1]} drank {str(items)[1:-1]} at location: {agent.location}')
                self.delete_thing(items)
        elif action == "play":
            items = self.list_things_at(agent.location, tclass=Bone)
            if items and agent.play_with_bone(items):
                print(f'{str(agent)[1:-1]} is playing with {str(items)[1:-1]} at location: {agent.location}')

    def is_done(self):
        no_edibles = not any(isinstance(thing, (Food, Water)) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles


# Set up environment and agents
park = Park2D(3, 3, color={
    'EnergeticBlindDog': (200, 0, 0),
    'Water': (0, 200, 200),
    'Food': (230, 115, 40),
    'Bone': (150, 75, 0)
})

dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
bone = Bone()

park.add_thing(bone, [2,1])
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])
park.add_thing(Water(), [1,0])
park.add_thing(Food(), [2,0])

print("Dog started at [0,0], facing down. Let's see what happens!")
park.run(20)


,,
,,
,,


## 2: Blind DOG

- Run and understand all the code above for the blind dog and the energetic blind dog.


In [24]:
from agents import Agent, Thing, Environment   # assuming from AIMA's agents.py

# ---------- Thing Classes ----------
class Food(Thing):
    pass

class Water(Thing):
    pass

class Bone(Thing):
    pass


# ---------- Park Environment ----------
class Park(Environment):
    def percept(self, agent):
        """Return list of Things at agent's location"""
        return self.list_things_at(agent.location)

    def execute_action(self, agent, action):
        if action == "move down":
            print(f'{str(agent)[1:-1]} decided to {action} at location: {agent.location}')
            agent.movedown()

        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if items:
                item = items[0]
                if agent.eat(item):
                    print(f'{str(agent)[1:-1]} ate {item} at location: {agent.location}')
                    self.delete_thing(item)

        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if items:
                item = items[0]
                if agent.drink(item):
                    print(f'{str(agent)[1:-1]} drank {item} at location: {agent.location}')
                    self.delete_thing(item)

        elif action == "play":
            items = self.list_things_at(agent.location, tclass=Bone)
            if items:
                item = items[0]
                if agent.play_with_bone(item):
                    print(f'{str(agent)[1:-1]} is playing with {item} at location: {agent.location}')
                    # Note: bone is not deleted – dog can play forever

    def is_done(self):
        no_edibles = not any(isinstance(thing, (Food, Water, Bone)) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

    def step(self):
        """Override step to avoid attribute errors"""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")

            for (agent, action) in zip(self.agents, actions):
                if action:
                    self.execute_action(agent, action)

            self.exogenous_change()
            print("Step complete for Park environment.\n")

    def delete_thing(self, thing):
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print(" in Environment delete_thing")
            print(f" Thing to be removed: {thing} at {thing.location}")
            print(f" from list: {[(obj, obj.location) for obj in self.things]}")
        if thing in self.agents:
            self.agents.remove(thing)


# ---------- BlindDog Agent ----------
class BlindDog(Agent):
    def __init__(self, program=None):
        super().__init__(program)
        self.has_bone = False   # Track if playing with bone

    # Location must be a tuple (x, y)
    @property
    def location(self):
        return getattr(self, "_location", (0, 1))  # default start if not set

    @location.setter
    def location(self, value):
        self._location = value

    def movedown(self):
        x, y = self.location
        self.location = (x, y + 1)

    def eat(self, thing):
        return isinstance(thing, Food)

    def drink(self, thing):
        return isinstance(thing, Water)

    def play_with_bone(self, thing):
        if isinstance(thing, Bone):
            self.has_bone = True
            print("Dog is happily playing with the bone!")
            return True
        return False


# ---------- EnergeticBlindDog with Stateful Program ----------
class EnergeticBlindDog(BlindDog):
    def __init__(self):
        super().__init__(program=self.program)
        self.eat_drink_count = 0
        self.just_ate_or_drank = False

    def program(self, percept):
        # If already has bone, always play
        if self.has_bone:
            print("Action: play")
            return "play"

        # If just ate or drank, move down
        if self.just_ate_or_drank:
            self.just_ate_or_drank = False
            print("Action: move down")
            return "move down"

        # Sense environment
        food_present = any(isinstance(thing, Food) for thing in percept)
        water_present = any(isinstance(thing, Water) for thing in percept)
        bone_present = any(isinstance(thing, Bone) for thing in percept)

        # Act
        if food_present:
            self.eat_drink_count += 1
            self.just_ate_or_drank = True
            print("Action: eat")
            return "eat"
        elif water_present:
            self.eat_drink_count += 1
            self.just_ate_or_drank = True
            print("Action: drink")
            return "drink"
        elif self.eat_drink_count >= 2:
            if bone_present:
                print("Action: play")
                return "play"
            else:
                print("Action: move down")
                return "move down"
        else:
            print("Action: move down")
            return "move down"


# ========== Setup and Run ==========
if __name__ == "__main__":
    park = Park()

    # Create dog agent
    dog = EnergeticBlindDog()
    dog.location = (0, 10)
    park.add_thing(dog, location=dog.location)

    # Add Food, Water, Bone at specified locations
    park.add_thing(Food(), location=(0, 11))
    park.add_thing(Water(), location=(0, 14))
    park.add_thing(Bone(), location=(0, 19))

    # Run environment
    park.run(steps=20)


Action: move down
EnergeticBlindDog decided to move down at location: (0, 10)
Step complete for Park environment.

Action: eat
EnergeticBlindDog ate <Food> at location: (0, 11)
Step complete for Park environment.

Action: move down
EnergeticBlindDog decided to move down at location: (0, 11)
Step complete for Park environment.

Action: move down
EnergeticBlindDog decided to move down at location: (0, 12)
Step complete for Park environment.

Action: move down
EnergeticBlindDog decided to move down at location: (0, 13)
Step complete for Park environment.

Action: drink
EnergeticBlindDog drank <Water> at location: (0, 14)
Step complete for Park environment.

Action: move down
EnergeticBlindDog decided to move down at location: (0, 14)
Step complete for Park environment.

Action: move down
EnergeticBlindDog decided to move down at location: (0, 15)
Step complete for Park environment.

Action: move down
EnergeticBlindDog decided to move down at location: (0, 16)
Step complete for Park enviro

In [25]:
park = Park2D(5,20, color={'BlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)}) # park width is set to 5, and height to 20
dog = BlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,1])
park.add_thing(dogfood, [0,5])
park.add_thing(water, [0,7])
morewater = Water()
park.add_thing(morewater, [0,15])
print("BlindDog starts at (1,1) facing downwards, lets see if he can find any food!")
park.run(20)

,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,
,,,,


AttributeError: 'BlindDog' object has no attribute 'direction'

In [ ]:
park = Park2D(5,5, color={'EnergeticBlindDog': (200,0,0), 'Water': (0, 200, 200), 'Food': (230, 115, 40)}) # You might need to change the park size
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [0,1])
morewater = Water()
morefood = Food()
park.add_thing(morewater, [2,4])
park.add_thing(morefood, [4,3])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")
park.run(20)

TypeError: EnergeticBlindDog.__init__() takes 1 positional argument but 2 were given

![Happy Dog](https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExemw5ODVtZG5yY2xicm1lYjExNHI3NDA1cGxpbXp4MGp3aXlpeGIwNiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/1Ju5mGZlWAqek/giphy.gif)